# Math 582 Miniproject 3 - Model Development

The purpose of this notebook is implment dual SVM convex quadratic optimization for the purposes of binary classification.

In [77]:
# Imports

import numpy as np
import pandas as pd
from functools import reduce
from qpsolvers import solve_qp
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [78]:
# read in data
df = pd.read_csv('./data/test-data/test_data.csv')
df

,age,interest,success
0,23.657801,18.859917,0.0
1,22.573729,17.969223,0.0
2,32.553424,29.463651,0.0
3,6.718035,25.704665,1.0
4,14.401919,16.770856,0.0
...,...,...,...
292,27.697220,18.799309,0.0
293,15.150959,72.000352,1.0
294,22.264378,68.453459,1.0
295,25.677420,90.118212,1.0


In [79]:
# This function performs the following:
# - maps classifiers values -1 or +1
# - separates feature columns from the classifier column
# - splits the data into training and testing sets
# returns: xs_train, xs_test, ys_train, ys_test

def prep_data(df, classifier_column_name, classifier_vals, train_size=0.75):

    if len(classifier_vals) != 2:
        raise ValueError("classifier_vals argument must be length 2 (binary classifier)")
    
    # map each binary classifier value to either 1 or -1
    df[classifier_column_name] = df[classifier_column_name].apply(lambda b: -1 if b == classifier_vals[0] else 1)

    # separate the features from the classifications
    colnames = df.columns.tolist()
    feature_column_names = list(filter(lambda colname: colname != classifier_column_name, colnames))

    xs = df[feature_column_names]
    ys = df[classifier_column_name]

    # split the data into training and testing data
    datasets = train_test_split(xs, ys, train_size=train_size)

    # map all the training data into numpy arrays
    xs_train, xs_test, ys_train, ys_test = list(map(lambda s: s.to_numpy(), datasets))

    # return the training and testing data
    return xs_train, xs_test, ys_train, ys_test


In [80]:
# compute the kernel matrix K
def gram_matrix(xs_train, k):
    N = xs_train.shape[0]
    K = np.zeros(shape=(N,N))
    for i in range(0, N):
        for j in range(0, i + 1):
            K[i][j] = K[j][i] = k(xs_train[i], xs_train[j])
    return K


In [81]:
def print_neg_eigvals(A):
    eigvals = np.linalg.eigvals(A)
    neg_eigvals = np.extract(eigvals < 0, eigvals)
    print("Found {} negative eigenvalues.\nNegative eigenvalues = {}".format(len(neg_eigvals), neg_eigvals))

# assume A is an NxN matrix
def make_positive_definite(A):
    N = A.shape[0]
    # if all eigenvals are not > 0, then add perturbation and try again
    while not np.all(np.linalg.eigvals(A) > 0):
        # print_neg_eigvals(A)
        epsilon = 1e-12
        perturbation = epsilon * np.identity(N)
        A += perturbation
    return A

In [82]:
# define the function used to get the optimal lagrange multipliers (alpha) for given training data, kernel function, and cost function
def lagrange_multipliers(xs_train, ys_train, k, C):
    # Solves for x in the following...
    # min 1/2 x^T P x + q^T x
    # s.t.
    #  Gx <= h

    N = xs_train.shape[0]

    # compute the entries in the kernel matrix
    K = gram_matrix(xs_train, k)
    # Y = np.diag(ys_train)

    # quadratic program parameters
    P = make_positive_definite(np.outer(ys_train,ys_train) * K)
    q = -1 * np.ones(N)
    G = np.vstack([ys_train, -1 * ys_train, -1 * np.identity(N), np.identity(N)])
    h = np.concatenate([np.zeros((N+2)), C * np.ones((N))])

    solution = solve_qp(P, q, G, h)
    return solution

In [83]:
def evaluate_classifier(classifier, xs_test, ys_test):
    numtests = xs_test.shape[0]
    results = classifier(xs_test) == ys_test
    numcorrect = len(list(filter(lambda b: b, list(results))))
    successrate = numcorrect / numtests
    return successrate

In [84]:
def build_svm_classifier(xs_train, ys_train, k, C):

    # anything less than this value is considered by us to be 0 when selecting support vectors
    MIN_SUPPORT_VECTOR_MULTIPLIER = 1e-5

    N = xs_train.shape[0]
    alpha = lagrange_multipliers(xs_train, ys_train, k, C)

    # select support multipliers that are > 0
    support_vector_indices = (alpha > MIN_SUPPORT_VECTOR_MULTIPLIER)
    support_multipliers = alpha[support_vector_indices]
    support_vectors = xs_train[support_vector_indices]
    support_classifications = ys_train[support_vector_indices]

    fw = lambda n: support_multipliers[n] * support_classifications[n] * support_vectors[n]
    w = np.array(list(reduce(lambda v1, v2: v1 + v2, [fw(n) for n in range(0, len(support_vectors))])))

    # select support multipliers that are > 0 AND < C (the cost)
    within_cost_indices = (support_multipliers < C)
    support_multipliers_ = support_multipliers[within_cost_indices]
    support_vectors_ = support_vectors[within_cost_indices]
    support_classifications_ = support_classifications[within_cost_indices]

    fb = lambda n: np.abs(support_classifications_[n] - np.inner(w, support_vectors_[n]))
    b = np.median(np.array([fb(n) for n in range(0, len(support_vectors_))]))

    # builds the classifier function
    # choose whether to invert the classifications of +1 / -1
    def classifier(invert=False):
        # classifies a single sample as +1 or -1
        def classify_sample(x_test):
            v = -1 if invert else 1
            return (v if np.inner(w, x_test) + b >= 0 else -1 * v)

        # classifies multiple samples as +1 or -1
        def classify_samples(xs_test):
            return np.array(list(map(classify_sample, list(xs_test))))

        return classify_samples

    # figure out whether to invert the classification identification (+1 or -1)
    # based on training data
    # invert if the success rate is less than 50% on training data
    invert = evaluate_classifier(classifier(invert=False), xs_train, ys_train) < 0.5

    # take 5 samples from 

    return classifier(invert=invert)


In [85]:
# Kernel functions

def linear_kernel():
    return np.inner

def poly_kernel(dim, offset=1.0):
    if dim < 1:
        raise ValueError("Invalid polynomial dimension for polynomial kernel")

    def f(x, y):
        return (offset + np.inner(x, y)) ** dim

    return f

def gaussian_kernel(sigma):
    def f(x, y):
        exponent = -np.sqrt(np.linalg.norm(x-y) ** 2 / (2 * sigma ** 2))
        return np.exp(exponent)
    return f

def hyperbolic_tangent_kernel(kappa, c):
    def f(x, y):
        return np.tanh(kappa * np.dot(x, y) + c)
    return f

In [86]:
xs_train, xs_test, ys_train, ys_test = prep_data(df, "success", [0.0, 1.0], train_size=0.75)

k = linear_kernel()
C = 1
classifier = build_svm_classifier(xs_train, ys_train, k, C)

In [87]:
evaluate_classifier(classifier, xs_test, ys_test)

0.84

In [88]:
linsvc = SVC(kernel=k, C=C)
linsvc.fit(xs_train, ys_train)
evaluate_classifier(linsvc.predict, xs_test, ys_test)

0.8266666666666667

In [89]:
classifier(xs_test) == ys_test

array([ True, False,  True,  True,  True,  True, False, False, False,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False, False,  True,  True,
        True,  True, False,  True,  True, False,  True,  True,  True,
       False,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True])

In [90]:
linsvc.predict(xs_test) == ys_test

array([ True, False,  True,  True,  True,  True, False, False, False,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False, False,  True,  True,
        True,  True, False,  True,  True, False,  True,  True,  True,
       False,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True])